Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Load Data

In [ ]:
import tensorflow as tf

In [ ]:
#loading the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Subset Class 0 and Class 1 from Data

In [ ]:
# subsetting for classes 0,1 only

x_train = x_train[np.logical_or(y_train == 0, y_train == 1)]
y_train = y_train[np.logical_or(y_train == 0, y_train == 1)]

x_test = x_test[np.logical_or(y_test == 0, y_test == 1)]
y_test = y_test[np.logical_or(y_test == 0, y_test == 1)]


In [ ]:
np.unique(y_train), np.unique(y_test)

(array([0, 1], dtype=uint8), array([0, 1], dtype=uint8))

In [ ]:
x_train=x_train.reshape(len(x_train),-1)

In [ ]:
x_test=x_test.reshape(len(x_test),-1)

In [ ]:
x_train.shape[0]

12665

Standardize Data

In [ ]:
x_train=(x_train-np.mean(x_train,axis=0))/(np.std(x_train,axis=0)+10e-16)
x_test=(x_test-np.mean(x_test,axis=0))/(np.std(x_test,axis=0)+10e-16)

Divide data into training and validation set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validation, Y_train, Y_validation = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
epochs=100
eta=0.1
tolerance = 10e-8
Lambda=[0.01,0.1]
#batch_size=10
batch_size=[10,20]
m = X_train.shape[0]
ACCURACY=[]
Name=[]

Sigmoid Function

In [ ]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

*Cost* Function with Lasso (L1) Regularization

In [ ]:
def cost_function(phiz, Y_train,w,Lambda):
    return ((Y_train * np.log(phiz) - (1 - Y_train) * np.log(1 - phiz)).mean())+0.5*Lambda*np.mean(np.abs(w))

*Cost* Function

In [ ]:
def cost_function(phiz, Y_train,w,Lambda):
    return ((Y_train * np.log(phiz) - (1 - Y_train) * np.log(1 - phiz)).mean())

Accuracy Function

In [ ]:
def accuracy(X_validation,w,b):
  z=np.dot(X_validation,w.T)+b
  y_pred =sigmoid(z)
  for i in range(len(y_pred)):
    if (y_pred[i]>0.5):
     y_pred[i]=1
    else:
      y_pred[i]=0
    accuracy=np.mean(np.equal(y_pred, Y_validation))*100

  return accuracy

Gradient Descent Function

In [ ]:
def Gradient_Descent(X_train,Y_train,w,b,epochs,eta,Lambda):
    for i in range (epochs):
          z=np.dot(X_train,w.T)+b
          phiZ = sigmoid(z)
          Cost[i] =cost_function(phiZ, Y_train,w,Lambda)
          w = w - (eta * (np.dot((phiZ - Y_train).T , X_train) /len(X_train)+Lambda/len(X_train)*np.sign(w)))
          b = b - eta * np.mean(phiZ - Y_train)
          if (Cost[i]<tolerance):
            break;

    return w,b

In [ ]:
for lamb in range(len(Lambda)):
  w=np.random.rand(784)
  b=np.random.rand(1)
  Cost=np.zeros(epochs)
  w,b=Gradient_Descent(X_train,Y_train,w,b,epochs,eta,Lambda[lamb])
  Accuracy=accuracy(X_validation,w,b)
  print('Accuracy with Lambda (',Lambda[lamb],') is =',Accuracy)

Accuracy with Lambda ( 0.01 ) is = 98.85511251480457
Accuracy with Lambda ( 0.1 ) is = 98.53928148440583


In [ ]:
Accuracy=accuracy(X_validation,w,b)
ACCURACY.append(Accuracy)
Name.append('Gradient_Descent')
Accuracy

98.53928148440583

Mini_Batch_Gradient_Descent Function

In [ ]:
def mini_batch_gradient_descent(X_train, Y_train,w,b,eta, Lambda, batch_size,k):
  for i in range(batch_size):
     if(i==0):
        num_batches=k
     xX_train=X_train[:num_batches]
     yY_train=Y_train[:num_batches]
     w,b=Gradient_Descent(X_train,Y_train,w,b,1,eta,Lambda)
     num_batches+=k
  return w,b
for i in range(len(batch_size)):
  k = int(m / batch_size[i])
  w=np.random.rand(784)
  b=np.random.rand(1)
  Cost=np.zeros(epochs)
  print('this is the ',i+1,'batch')
  for lamb in range(len(Lambda)):
    for j in range(epochs):
      w,b= mini_batch_gradient_descent(X_train, Y_train,w,b,eta, Lambda[lamb], batch_size[i],k)
      z=np.dot(X_train,w.T)+b
      phiZ = sigmoid(z)
      Cost[j] =cost_function(phiZ, Y_train,w,Lambda[lamb])
    Accuracy=accuracy(X_validation,w,b)
    print('Accuracy with Lambda (',Lambda[lamb],') is =',Accuracy)

this is the  1 batch
Accuracy with Lambda ( 0.01 ) is = 99.72364784840111
Accuracy with Lambda ( 0.1 ) is = 99.64469009080142
this is the  2 batch
Accuracy with Lambda ( 0.01 ) is = 99.68416896960126
Accuracy with Lambda ( 0.1 ) is = 99.64469009080142


In [ ]:
Accuracy=accuracy(X_validation,w,b)
ACCURACY.append(Accuracy)
Name.append('Mini_Batch_Gradient_Descent')
Accuracy

99.64469009080142

RMD_PROP Optimizer Function

In [ ]:
def rms_prop(X_train, Y_train, w, b, eta, Lambda, beta, epsilon):
  z=np.dot(X_train,w.T)+b
  phiZ = sigmoid(z)
  dW = (np.dot((phiZ - Y_train).T , X_train) /len(X_train)+Lambda/len(X_train)*w)
  db=np.mean(phiZ - Y_train)
  vdw = beta*0+(1 - beta) * np.power(dW, 2)
  vdb=beta*0+(1 - beta) * np.power(db, 2)
  w= w-(eta / np.sqrt(vdw + epsilon)) * dW
  b= b-(eta / np.sqrt(vdb + epsilon)) * db
  return w,b

w=np.random.rand(784)
b=np.random.rand(1)
Cost=np.zeros(epochs)
beta=0.9
epsilon=10e-8
for lamb in range(len(Lambda)):
  for i in range(epochs):
    w,b=rms_prop(X_train, Y_train, w,b, eta, Lambda[lamb], beta, epsilon)
    z=np.dot(X_train,w.T)+b
    phiZ = sigmoid(z)
    Cost[i] =cost_function(phiZ, Y_train,w,Lambda[lamb])
  Accuracy=accuracy(X_validation,w,b)
  print('Accuracy with Lambda (',Lambda[lamb],') is =',Accuracy)


Accuracy with Lambda ( 0.01 ) is = 99.76312672720094
Accuracy with Lambda ( 0.1 ) is = 99.8026056060008


In [ ]:
Accuracy=accuracy(X_validation,w,b)
ACCURACY.append(Accuracy)
Name.append('RMSPROP Optimizer')
Accuracy

99.8026056060008

ADAM Oprimizer Function

In [ ]:
def adam(X_train, Y_train, w,b, eta, Lambda, beta1, beta2, epsilon, t):
  z=np.dot(X_train,w.T)+b
  phiZ = sigmoid(z)
  dW = (np.dot((phiZ - Y_train).T , X_train) /len(X_train)+Lambda/len(X_train)*w)
  db=np.mean(phiZ - Y_train)
  m_w = beta1 * 0 + (1 - beta1) * dW
  v_w = beta2 * 0 + (1 - beta2) * np.power(dW, 2)
  m_b = beta1 * 0 + (1 - beta1) * db
  v_b = beta2 * 0 + (1 - beta2) * np.power(db, 2)
  m_hat_w=m_w/(1-np.power(beta1,t))
  v_hat_w=v_w/(1-np.power(beta2,t))
  m_hat_b=m_b/(1-np.power(beta1,t))
  v_hat_b=v_b/(1-np.power(beta2,t))
  w=w-(eta/np.sqrt(v_hat_w)+epsilon)*m_hat_w
  b=b-(eta/np.sqrt(v_hat_b)+epsilon)*m_hat_b
  return w,b

w=np.random.rand(784)
b=np.random.rand(1)
Cost=np.zeros(epochs)
beta1=0.9
beta2=0.999
epsilon=10e-8
t=1
for lamb in range(len(Lambda)):
  for i in range(epochs):
    w,b=adam(X_train, Y_train, w,b, eta,Lambda[lamb], beta1,beta2,epsilon,t=i+1)
    z=np.dot(X_train,w.T)+b
    phiZ = sigmoid(z)
    Cost[i] =cost_function(phiZ, Y_train,w,Lambda[lamb])
  Accuracy=accuracy(X_validation,w,b)
  print('Accuracy with Lambda (',Lambda[lamb],') is =',Accuracy)

Accuracy with Lambda ( 0.01 ) is = 99.60521121200158
Accuracy with Lambda ( 0.1 ) is = 99.5262534544019


In [ ]:
Accuracy=accuracy(X_validation,w,b)
ACCURACY.append(Accuracy)
Name.append('Adam Optimizer')
Accuracy

99.5262534544019

In [ ]:
ACCURACY

[98.53928148440583, 99.64469009080142, 99.8026056060008, 99.5262534544019]

In [ ]:
# Test with same data and look at the accuracy
z=np.dot(x_test,w.T)+b
y_pred =sigmoid(z)
for i in range(len(y_pred)):
  if (y_pred[i]>0.5):
     y_pred[i]=1
  else:
     y_pred[i]=0


accuracy=np.mean(np.equal(y_pred, y_test))*100
accuracy

99.62174940898345

In [ ]:
name=pd.DataFrame(Name,columns=['Optimizer'])
ACCuracy=pd.DataFrame(ACCURACY,columns=['Accuracy'])

In [ ]:
Table=pd.concat([name,ACCuracy],axis=1)
Table

,Optimizer,Accuracy
0,Gradient_Descent,98.539281
1,Mini_Batch_Gradient_Descent,99.644690
2,RMSPROP Optimizer,99.802606
3,Adam Optimizer,99.526253


Use L1 regularization with gradient descent optimizer: This involves adding an L1 penalty term to the loss function for your logistic regression model, which encourages sparsity in the model weights. You can train this regularized model using gradient descent optimization, with a learning rate and regularization strength hyperparameters.



Use mini-batch gradient descent optimizer: This involves dividing your dataset into multiple batches, and updating the model weights using the gradients computed on each batch. You can train this model using a variety of batch sizes and learning rates, and evaluate its performance on the validation set.


Use RMSProp optimizer and Adam optimizer: These are both adaptive optimization algorithms that adjust the learning rate based on the magnitude of the gradients and the history of the parameter updates. You can train your logistic regression model using either of these optimizers, and tune the hyperparameters to obtain the best performance on the validation set.

L1 regularization with gradient descent optimizer: The L1 regularization can help to reduce overfitting and improve the generalization performance of the model. The choice of the regularization strength hyperparameter can affect the sparsity of the model weights and its performance on the validation set.

Mini-batch gradient descent optimizer: The mini-batch gradient descent can speed up the optimization process and reduce the variance of the gradient estimates. The choice of the batch size hyperparameter can affect the convergence rate and stability of the optimization algorithm.

RMSProp optimizer: The RMSProp optimizer can adaptively adjust the learning rate based on the magnitude of the gradients and the history of the parameter updates. It can converge faster than gradient descent and mini-batch gradient descent for certain types of problems.

Adam optimizer: The Adam optimizer combines ideas from momentum and RMSProp and is considered a state-of-the-art optimization algorithm. It can converge faster than other optimization algorithms, particularly for large datasets and complex models. However, it may be computationally expensive due to the additional computation required for maintaining the moving averages and may not always converge to the optimal solution.